### Data collection to label the offshores wind-parcs of Germany on the map

In [7]:
import pandas as pd
import numpy as np
import re

In [3]:
link = "https://de.wikipedia.org/wiki/Liste_der_deutschen_Offshore-Windparks"

offshores = pd.read_html(link,header=0)[0]



offshores.head(5)

,Name,Leistung (MW),"Windkraftanlagentyp (Leistung, Rotordurchmesser)",Anzahl WKAs,Koordinaten,Status,Inbetrieb­nahme (Jahr),"Quellen, Anmerkungen"
0,Nordsee,Nordsee,Nordsee,Nordsee,Nordsee,Nordsee,Nordsee,Nordsee
1,alpha ventus,6048,"6 × REpower 5M (5,08 MW, 126,5 m) 6 × AREVA M5...",12,"54° 0′ 30″ N, 6° 35′ 54″ O",in Betrieb,2010,NaN
2,BARD Offshore I,400,"BARD 5.0 (5,0 MW, 122,0 m)",80,"54° 21′ 30″ N, 5° 58′ 30″ O",in Betrieb,2013,NaN
3,Riffgat,1134,"Siemens SWT-3.6-120 (3,78 MW, 120,0 m)",30,"53° 41′ 24″ N, 6° 28′ 48″ O",in Betrieb,2014,[5]
4,Amrumbank West,3024,"Siemens SWT-3.6-120 (3,78 MW, 120,0 m)",80,"54° 30′ 0″ N, 7° 48′ 0″ O",in Betrieb,2015,[6][7]


In [4]:
#The coorinates ('Kkordinaten') will be split for two variablles
#Latitude and Longitude

fn = lambda x: pd.Series([i for i in (x.split(','))])
coord = offshores['Koordinaten'].apply(fn)
coord.rename(columns={0:'Latitude',1:'Longitude'},inplace=True)
offshores['Latitude']=coord['Latitude']
offshores['Longitude']=coord['Longitude']


offshores.info()
#we see the presence some NaNs 
#it is because the Windparks are 'In Planung'
#One of the possibilities is to remove them from dataset,
#Second is to label them as well (this another idea ) and 
#to label them in differnet colours on the map 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 10 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   Name                                              47 non-null     object
 1   Leistung (MW)                                     47 non-null     object
 2   Windkraftanlagentyp (Leistung, Rotordurchmesser)  39 non-null     object
 3   Anzahl WKAs                                       38 non-null     object
 4   Koordinaten                                       47 non-null     object
 5   Status                                            47 non-null     object
 6   Inbetrieb­nahme (Jahr)                            47 non-null     object
 7   Quellen, Anmerkungen                              42 non-null     object
 8   Latitude                                          47 non-null     object
 9   Longitude                         

In [12]:
#I removed them so far 
#As well as the rows woth Headers: Nordesee and Ostsee
offshores.drop(offshores.loc[offshores['Status']=='in Planung'].index, inplace=True)
offshores.drop(offshores.loc[offshores['Status']=='Nordsee'].index, inplace=True)
offshores.drop(offshores.loc[offshores['Status']=='Ostsee'].index, inplace=True)
offshores.drop(offshores.loc[offshores['Status']=='Planung eingestellt'].index, inplace=True)

In [13]:
offshores.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33 entries, 1 to 42
Data columns (total 12 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Name                                              33 non-null     object 
 1   Leistung (MW)                                     33 non-null     object 
 2   Windkraftanlagentyp (Leistung, Rotordurchmesser)  33 non-null     object 
 3   Anzahl WKAs                                       33 non-null     object 
 4   Koordinaten                                       33 non-null     object 
 5   Status                                            33 non-null     object 
 6   Inbetrieb­nahme (Jahr)                            33 non-null     object 
 7   Quellen, Anmerkungen                              28 non-null     object 
 8   Latitude                                          33 non-null     object 
 9   Longitude                   

In [14]:
#The Latitude and Longitude are in the format which is not recognised by Folium
#I have to bring it to decimal format

offshores['Latitude'] = offshores['Latitude'].str.split('N', n=1).str.get(0)
offshores['Longitude'] = offshores['Longitude'].str.split('O', n=1).str.get(0)


offshores['Latitude'] = offshores['Latitude'].str.split('″', n=1).str.get(0)
offshores['Longitude'] = offshores['Longitude'].str.split('″', n=1).str.get(0)

Latitude_cor=[]

for lat in offshores['Latitude']:
    deg, minutes, seconds =  re.split('[°\′]', lat)
    Latitude_cor.append(float(deg) + float(minutes)/60 + float(seconds)/(60*60))
    
Longitude_cor=[]

for long in offshores['Longitude']:
    deg, minutes, seconds =  re.split('[°\′]', long)
    Longitude_cor.append(float(deg) + float(minutes)/60 + float(seconds)/(60*60))
    
len(Latitude_cor)

33

In [15]:
offshores['Latitude_cor']=Latitude_cor
offshores['Longitude_cor']=Longitude_cor
offshores.head(3)

,Name,Leistung (MW),"Windkraftanlagentyp (Leistung, Rotordurchmesser)",Anzahl WKAs,Koordinaten,Status,Inbetrieb­nahme (Jahr),"Quellen, Anmerkungen",Latitude,Longitude,Latitude_cor,Longitude_cor
1,alpha ventus,6048,"6 × REpower 5M (5,08 MW, 126,5 m) 6 × AREVA M5...",12,"54° 0′ 30″ N, 6° 35′ 54″ O",in Betrieb,2010,NaN,54° 0′ 30,6° 35′ 54,54.008333,6.598333
2,BARD Offshore I,400,"BARD 5.0 (5,0 MW, 122,0 m)",80,"54° 21′ 30″ N, 5° 58′ 30″ O",in Betrieb,2013,NaN,54° 21′ 30,5° 58′ 30,54.358333,5.975000
3,Riffgat,1134,"Siemens SWT-3.6-120 (3,78 MW, 120,0 m)",30,"53° 41′ 24″ N, 6° 28′ 48″ O",in Betrieb,2014,[5],53° 41′ 24,6° 28′ 48,53.690000,6.480000


In [16]:
offshores_final=offshores[['Name', 'Anzahl WKAs', 'Latitude_cor', 'Longitude_cor','Status']]
offshores_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33 entries, 1 to 42
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Name           33 non-null     object 
 1   Anzahl WKAs    33 non-null     object 
 2   Latitude_cor   33 non-null     float64
 3   Longitude_cor  33 non-null     float64
 4   Status         33 non-null     object 
dtypes: float64(2), object(3)
memory usage: 1.5+ KB


In [18]:
offshores_final.to_csv('Offshores_Windparks.csv')

In [23]:
df=pd.read_csv('Offshores_Windparks.csv')

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     33 non-null     int64  
 1   Name           33 non-null     object 
 2   Anzahl WKAs    33 non-null     int64  
 3   Latitude_cor   33 non-null     float64
 4   Longitude_cor  33 non-null     float64
 5   Status         33 non-null     object 
dtypes: float64(2), int64(2), object(2)
memory usage: 1.7+ KB
